In [69]:
import sys
import os
import pandas as pd

# Add the root directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [70]:
from supabase_backend import create_supabase_connection, insert_data_into_table
from supabase_helper_functions import prepare_data_music_tracks

In [71]:
supabase_client = await create_supabase_connection()

Supabase connection created:  <supabase._sync.client.SyncClient object at 0x336993130>


In [91]:
df = pd.read_csv("/Users/pratikhotchandani/Downloads/Github/DS_5500/data_collection/cleaned_data/final_cleaned_data/output3.csv")

/var/folders/dq/pg0mssxn6yzdwtcdrwnt0bs80000gn/T/ipykernel_96794/2934980671.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/pratikhotchandani/Downloads/Github/DS_5500/data_collection/cleaned_data/final_cleaned_data/output3.csv")


In [92]:
# Function to safely convert values to integers with range validation
def convert_key(val):
    try:
        val = int(float(val))  # Convert to float first to handle cases like "5.0", then to int
        if 0 <= val <= 11:     # Ensure the value is within the valid range for "key"
            return val
    except (ValueError, TypeError):
        pass
    return None  # Return None for invalid or out-of-range values

def convert_mode(val):
    try:
        if isinstance(val, str):  # Handle string representations of "Major" and "Minor"
            val = val.strip().lower()
            if val == 'major':
                return 1
            elif val == 'minor':
                return 0
        val = int(float(val))  # Convert to float first to handle cases like "1.0", then to int
        if val in [0, 1]:       # Ensure the value is either 0 or 1
            return val
    except (ValueError, TypeError):
        pass
    return None  # Return None for invalid or out-of-range values

# Apply the conversion functions to the respective columns
df['key'] = df['key'].apply(convert_key)
df['mode'] = df['mode'].apply(convert_mode)

# Convert columns to nullable integer type (Int64) to ensure they are stored as integers with support for NaN
df['key'] = df['key'].astype(pd.Int64Dtype())
df['mode'] = df['mode'].astype(pd.Int64Dtype())

In [85]:
df

,Unnamed: 0,artist_name,track_name,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,duration_ms,genres,id,unique_id,key,mode
0,0,Gen Hoshino,Comedy,0.676,0.4610,-6.746,0.1430,0.0322,0.000001,0.3580,0.715,230666.0,acoustic,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino - Comedy,1,0
1,1,Ben Woodward,Ghost - Acoustic,0.420,0.1660,-17.235,0.0763,0.9240,0.000006,0.1010,0.267,149610.0,acoustic,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward - Ghost - Acoustic,1,1
2,2,"Ingrid Michaelson, ZAYN",To Begin Again,0.438,0.3590,-9.734,0.0557,0.2100,0.000000,0.1170,0.120,210826.0,acoustic,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN - To Begin Again,0,1
3,3,Kina Grannis,Can't Help Falling In Love,0.266,0.0596,-18.515,0.0363,0.9050,0.000071,0.1320,0.143,201933.0,acoustic,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis - Can't Help Falling In Love,0,1
4,4,Chord Overstreet,Hold On,0.618,0.4430,-9.681,0.0526,0.4690,0.000000,0.0829,0.167,198853.0,acoustic,5vjLSffimiIP26QG5WcN2K,Chord Overstreet - Hold On,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1788227,1946332,Dirty Harry,Tsiou,0.875,0.7070,-4.824,0.3450,0.5300,0.000000,0.1130,0.612,85161.0,NaN,4ktTjOAAb4nYUSWDunxMeC,Dirty Harry - Tsiou,10,0
1788228,1946333,Nino Xypolitas,Mexri Telous,0.717,0.6840,-7.495,0.1450,0.1810,0.000011,0.1660,0.581,243428.0,NaN,5t4ea9v5BSCWNw0aMzUoLd,Nino Xypolitas - Mexri Telous,5,0
1788229,1946334,"Viktor Sheen, Calin",Virtuální drogy,0.665,0.7810,-8.070,0.0722,0.2200,0.000000,0.1070,0.201,214736.0,NaN,7E8LK4R9gAkSG4CD1kqvsX,"Viktor Sheen, Calin - Virtuální drogy",10,0
1788230,1946335,Luan Santana,AMBIENTE ERRADO - Ao Vivo,0.484,0.8450,-5.347,0.0378,0.0368,0.000000,0.3410,0.452,144307.0,NaN,0SWDToufOxRVIngk8c1oZi,Luan Santana - AMBIENTE ERRADO - Ao Vivo,2,1


In [93]:
df = df.drop(columns=['Unnamed: 0','unique_id'])

In [95]:
import pandas as pd

def split_dataframe_into_chunks(df, chunk_size=50000, output_prefix="chunk"):
    """
    Splits a DataFrame into smaller chunks and saves them as CSV files.

    :param df: Input pandas DataFrame.
    :param chunk_size: Number of rows per chunk.
    :param output_prefix: Prefix for the output chunk files.
    """
    num_chunks = (len(df) // chunk_size) + (1 if len(df) % chunk_size != 0 else 0)

    for i in range(num_chunks):
        start_row = i * chunk_size
        end_row = start_row + chunk_size
        chunk = df.iloc[start_row:end_row]
        
        output_file = f"{output_prefix}_{i+1}.csv"
        chunk.to_csv(output_file, index=False)
        print(f"Saved: {output_file}")



In [96]:
split_dataframe_into_chunks(df)

Saved: chunk_1.csv
Saved: chunk_2.csv
Saved: chunk_3.csv
Saved: chunk_4.csv
Saved: chunk_5.csv
Saved: chunk_6.csv
Saved: chunk_7.csv
Saved: chunk_8.csv
Saved: chunk_9.csv
Saved: chunk_10.csv
Saved: chunk_11.csv
Saved: chunk_12.csv
Saved: chunk_13.csv
Saved: chunk_14.csv
Saved: chunk_15.csv
Saved: chunk_16.csv
Saved: chunk_17.csv
Saved: chunk_18.csv
Saved: chunk_19.csv
Saved: chunk_20.csv
Saved: chunk_21.csv
Saved: chunk_22.csv
Saved: chunk_23.csv
Saved: chunk_24.csv
Saved: chunk_25.csv
Saved: chunk_26.csv
Saved: chunk_27.csv
Saved: chunk_28.csv
Saved: chunk_29.csv
Saved: chunk_30.csv
Saved: chunk_31.csv
Saved: chunk_32.csv
Saved: chunk_33.csv
Saved: chunk_34.csv
Saved: chunk_35.csv
Saved: chunk_36.csv


In [87]:
def format_genres(value):
    if isinstance(value, str):  
        return [value]  # Convert single string to a list
    elif isinstance(value, list):
        return value  # Already in correct format
    else:
        return []  # Default to empty list if value is invalid


In [88]:
prepared_music_data = df.to_dict(orient="records")

In [74]:
prepared_music_data = prepare_data_music_tracks(df)

In [89]:
# Apply the fix to all records
for record in prepared_music_data:
    record["genres"] = format_genres(record["genres"])

In [90]:
response_insert = await insert_data_into_table(supabase_client, "music_tracks", prepared_music_data, batch_size=100)


Inserted batch into table: music_tracks
Batch size: 100
Inserted batch into table: music_tracks
Batch size: 100
Inserted batch into table: music_tracks
Batch size: 100
Inserted batch into table: music_tracks
Batch size: 100
Inserted batch into table: music_tracks
Batch size: 100
Inserted batch into table: music_tracks
Batch size: 100
Inserted batch into table: music_tracks
Batch size: 100
Inserted batch into table: music_tracks
Batch size: 100
Inserted batch into table: music_tracks
Batch size: 100
Inserted batch into table: music_tracks
Batch size: 100
Inserted batch into table: music_tracks
Batch size: 100
Inserted batch into table: music_tracks
Batch size: 100
Inserted batch into table: music_tracks
Batch size: 100
Inserted batch into table: music_tracks
Batch size: 100
Inserted batch into table: music_tracks
Batch size: 100
Inserted batch into table: music_tracks
Batch size: 100
Inserted batch into table: music_tracks
Batch size: 100
Inserted batch into table: music_tracks
Batch si

ValueError: Out of range float values are not JSON compliant